Pobieranie wiadomości z bankier.pl
Zapis do pliku ./data/bankier_{timestamp}.txt

In [1]:
# ! pip install bs4
# ! pip install requests
# ! pip install tqdm
# ! pip install pandas
# ! pip install pyarrow
# ! pip install qgrid
# ! pip install blaze
# ! pip install ipynbname

In [2]:
import requests
from bs4 import BeautifulSoup
from tqdm import tqdm, trange
from datetime import datetime, timezone
from common.webarticle import WebArticle
from common.pl_helper import PolishLocaleHelper
import pandas as pd
import logging
import ipynbname

In [3]:
# settings:
portal_name = 'bankier'
website = 'https://www.bankier.pl/wiadomosc'
website_short = 'https://www.bankier.pl'
pagination_attributes = '/'
no_articles_to_scrap = 300
data_dir = '../data'
logging.basicConfig(
    filename=f'../logs/{ipynbname.name()}.ipynb.log',
    encoding='utf-8',
    level=logging.DEBUG
)
ignored_paragraph_starts = []

Sprawdźmy plik robots.txt, w czasie pisania tego kodu pozwalał na pobieranie plików z portalu bankier z kategorii wiadomosc z drobnymi wyjątkami (zakaz dotyczył kilku starych artykułów)

In [4]:
resp = requests.get('https://www.bankier.pl/robots.txt')
print(resp.text)

User-agent: *
Disallow: /content
Disallow: /e/
Disallow: /forum/zgloszenie
Disallow: /forum/wyslij-znajomemu
Disallow: /forum/twoje-posty
Disallow: /forum/obserwowane
Disallow: /forum/obserwuj
Disallow: /cf/viasms.html
Disallow: /gielda/notowania/akcje/last_intraday?symbol=
Disallow: /partnerzy
Disallow: /forum/nowa-wiadomosc
Disallow: /kredyty-hipoteczne/cenatorium
Disallow: /gielda/notowania/akcje/charts
Disallow: /wiadomosc/Australia-zdecyduje-o-przyszlosci-JSW-7539353.html
Disallow: /forum/temat_wojciech-sobieraj-nowym-prezesem-idea-bank,24967583.html
Disallow: /forum/temat_re-wojciech-sobieraj-nowym-prezesem-idea-bank,27082265.html
Disallow: /forum/temat_uwaga-ewelina-sokolowska-oszustka-mieszkaniowa,26040125.html
Disallow: /wiadomosc/Feerum-ocenia-pozew-Krot-Plon-o-zaplate-44-84-mln-zl-za-bezzasadny-7646880.html
Disallow: /forum/temat_powiedzmy-wprost-arkadiusz-kuich-to-oszust,11918677.html
Disallow: /forum/temat_ziemowit-stempin-strzezcie-sie-go,6797506.html
Disallow: /static/js

In [5]:
def check_if_ignored_paragraph(text, ignored_paragraphs, operation='startswith'):

    for ignored_paragraph in ignored_paragraphs:
        if operation == 'startswith':
            if text.startswith(ignored_paragraph):
                return True
        else:
            logging.debug(f'check_if_ignored: operation {operation} not implemented')
    return False


In [6]:
def scrap_article_text_bankier(link):
    logging.info('scrap_article_text_bankier:')
    logging.info(f'article link to scrap: {link}')

    page = requests.get(link)
    if page.status_code != 200:
        logging.debug(f'received status code: {page.status_code}')
        return '', ''

    soup = BeautifulSoup(page.content, 'html.parser')
    paragraphs = soup.find('section', class_='o-article-content').find_all('p')

    text = ''
    lead_text_int = ''

    for paragraph in paragraphs[:-1]:
        paragraph_text = paragraph.get_text().strip()
        if paragraph == '' or paragraph is None:
            continue
        if paragraph.find('span', class_='lead'):
            lead_text_int = paragraph_text
            continue
        if check_if_ignored_paragraph(paragraph_text, ignored_paragraph_starts):
            continue
        text += paragraph_text + ' '

    time_string = soup.find('div', class_='m-article-attributes').find('span', class_='a-span').get_text().strip()
    when_published = datetime.strptime(time_string, "%Y-%m-%d %H:%M")

    title_int = soup.find('h1', class_='a-heading -blue').get_text().strip()

    author = ''
    author_candidate = paragraphs[-1].get_text(strip=True)
    if len(author_candidate) <= 12:
        author = author_candidate
    else:
        text += author_candidate

    source_candidate = soup.find('span', class_='o-article-source__name')
    if source_candidate is None:
        source = 'Bankier.pl'
    else:
        source = source_candidate.get_text().strip()

    return text, when_published, title_int, lead_text_int, author, source

In [7]:
def scrap_articles_bankier(website):
    logging.info('scrap_articles_bankier:')
    logging.info(f'page with articles to scrap: {website}')

    result = list()
    page = requests.get(website)
    if page.status_code != 200:
        logging.debug(f'received status code: {page.status_code}')
        return result

    soup = BeautifulSoup(page.content, 'html.parser')
    page_content = soup.find('section',id='articleList').find_all('div',class_='article')

    print(f'articles to scrap: {len(page_content)}')

    for article in page_content:

        logging.info(f"article scrapping started: {article}")
        title = article.find('span',class_='entry-title').get_text().strip()
        lead_text = article.find('p').get_text().strip()
        link = website_short + article.find('span',class_='entry-title').find('a', href=True, rel='bookmark')['href']
        webart = WebArticle(title, lead_text, link, *scrap_article_text_bankier(link), portal_name)
        logging.info(f'webarticle scrapped: {webart}')
        result.append(webart)

    return result

In [8]:
no_scrapped_articles = 0
page_no=1
scrapped_articles = list()

logging.info(f'no_articles_to_scrap: {no_articles_to_scrap}')

pbar = tqdm(total=no_articles_to_scrap)
while no_scrapped_articles < no_articles_to_scrap:
    logging.info('next page: '+website+pagination_attributes+str(page_no))
    new_articles = scrap_articles_bankier(website+pagination_attributes+str(page_no))
    page_size = len(new_articles)
    logging.info(f'new_articles: {page_size}')
    if no_scrapped_articles + page_size > no_articles_to_scrap:
        page_size = no_articles_to_scrap - no_scrapped_articles
    scrapped_articles.extend(new_articles[:page_size])
    logging.info(f'scrapped new articles: {page_size}')
    no_scrapped_articles += page_size
    pbar.update(page_size)
    logging.info(f'no_scrapped_articles (total): {no_scrapped_articles}')
    page_no += 1
    if len(scrapped_articles) <= 0:
        logging.debug(f'0 articles scrapped!')
        break
pbar.close()

  0%|          | 0/300 [00:00<?, ?it/s]

articles to scrap: 10


  3%|▎         | 10/300 [00:01<00:56,  5.17it/s]

articles to scrap: 10


  7%|▋         | 20/300 [00:03<00:52,  5.35it/s]

...

 93%|█████████▎| 280/300 [00:57<00:03,  5.13it/s]

articles to scrap: 10


 97%|█████████▋| 290/300 [00:59<00:01,  5.22it/s]

articles to scrap: 10


100%|██████████| 300/300 [01:01<00:00,  4.90it/s]


In [9]:
for i in trange(len(scrapped_articles)):
    # print(f'{i}: {scrapped_articles[i].lead_text}')
    print(f'{i}: {len(scrapped_articles[i].text)}')

100%|██████████| 300/300 [00:00<00:00, 18039.50it/s]

0: 1863
1: 2312
2: 1431
3: 1668
4: 665
5: 1082
6: 4605
7: 134
8: 4550
9: 1169
10: 305
11: 311
12: 1081
13: 1747
14: 2399
15: 1237
16: 1938
17: 3271
18: 1465
19: 2512
20: 3468
21: 682
22: 1578
23: 1368
24: 426
25: 1138
26: 1204
27: 624
28: 3187
29: 597
30: 3262
31: 1741
32: 3443
33: 1221
34: 1516
35: 2790
36: 1540
37: 3255
38: 4638
39: 2512
40: 3081
41: 2825
42: 1779
43: 1181
44: 2128
45: 3885
46: 1486
47: 3064
48: 867
49: 3735
50: 4200
51: 2570
52: 3450
53: 2047
54: 894
55: 4799
56: 1341
57: 1911
58: 2045
59: 895
60: 2443
61: 1104
62: 2644
63: 5751
64: 2349
65: 2108
66: 5225
67: 1292
68: 5823
69: 873
70: 4521
71: 3402
72: 2968
73: 5328
74: 2318
75: 2521
76: 1584
77: 1943
78: 5812
79: 1649
80: 916
81: 2253
82: 1661
83: 1474
84: 5111
85: 1564
86: 2481
87: 3219
88: 3299
89: 1892
90: 2094
91: 3287
92: 2945
93: 1062
94: 1098
95: 495
96: 2818
97: 2393
98: 5337
99: 2071
100: 1487
101: 4396
102: 6102
103: 4972
104: 5440
105: 2053
106: 3095
107: 2162
108: 4545
109: 7542
110: 2393
111: 2361
112:

In [10]:
bankier_dataframe = pd.DataFrame(article.__dict__ for article in scrapped_articles)
bankier_dataframe

,title,title_int,lead_text,lead_text_int,link,text,when_published,author,source,portal
0,Spadki na Wall Street koniec sesji. Nasdaq znó...,Spadki na Wall Street koniec sesji. Nasdaq znó...,Środowa sesja na Wall Street zakończyła się sp...,Środowa sesja na Wall Street zakończyła się sp...,https://www.bankier.pl/wiadomosc/Wall-Street-w...,Dow Jones Industrial na zamknięciu spadł o 339...,2022-01-19 22:12:00,kkr/ pr/,PAP Biznes,bankier
1,Specjaliści od cyberbezpieczeństwa w administr...,Specjaliści od cyberbezpieczeństwa w administr...,Rada Ministrów przyjęła w środę rozporządzenie...,Rada Ministrów przyjęła w środę rozporządzenie...,https://www.bankier.pl/wiadomosc/Specjalisci-o...,"Jak wyjaśniono, rozporządzenie określa zasady ...",2022-01-19 20:18:00,pif/ mk/,PAP,bankier
2,Mark Brzezinski oficjalnie zaprzysiężony na am...,Mark Brzezinski oficjalnie zaprzysiężony na am...,Wiceprezydent USA Kamala Harris oficjalnie zap...,Wiceprezydent USA Kamala Harris oficjalnie zap...,https://www.bankier.pl/wiadomosc/Mark-Brzezins...,Podczas ceremonii w budynku Eisenhowera mieszc...,2022-01-19 20:17:00,osk/ tebe/,PAP,bankier
3,Portugalia wszczyna śledztwo w sprawie przyzna...,Portugalia wszczyna śledztwo w sprawie przyzna...,Prokuratura generalna Portugalii wszczęła w śr...,Prokuratura generalna Portugalii wszczęła w śr...,https://www.bankier.pl/wiadomosc/Portugalia-ws...,"Prokuratura generalna wyjaśniła, że przedmiote...",2022-01-19 20:11:00,zat/ mal/,PAP,bankier
4,Szef MSZ Zbigniew Rau zakażony koronawirusem,Szef MSZ Zbigniew Rau zakażony koronawirusem,Minister spraw zagranicznych Zbigniew Rau uzys...,Minister spraw zagranicznych Zbigniew Rau uzys...,https://www.bankier.pl/wiadomosc/Szef-MSZ-Zbig...,"""Z uwagi na pozytywny wynik testu na SARS-CoV-...",2022-01-19 20:05:00,wni/ mok/,PAP,bankier
...,...,...,...,...,...,...,...,...,...,...
295,Wyniki najnowszej ankiety NBP wskazują na znac...,Wyniki najnowszej ankiety NBP wskazują na znac...,Ankietowane przez NBP ośrodki analityczne spod...,Ankietowane przez NBP ośrodki analityczne spod...,https://www.bankier.pl/wiadomosc/Ankieta-NBP-w...,"""W najnowszej rundzie ankiety eksperci AM NBP ...",2022-01-17 10:10:00,pat/ asa/,PAP Biznes,bankier
296,Słabszy dolar sprzyja złotemu. Kurs euro bliże...,Słabszy dolar sprzyja złotemu. Kurs euro bliże...,"Kurs euro rozpoczął nowy tydzień od spadku, ce...",Obserwowane od początku stycznia osłabienie do...,https://www.bankier.pl/wiadomosc/Slabszy-dolar...,W ubiegłym tygodniu doszło do ważnych rozstrzy...,2022-01-17 10:01:00,KK,Bankier.pl,bankier
297,Credit Agricole zrewidował w górę prognozę inf...,Credit Agricole zrewidował w górę prognozę inf...,Ekonomiści Credit Agricole BP zrewidowali swoj...,Ekonomiści Credit Agricole BP zrewidowali swoj...,https://www.bankier.pl/wiadomosc/Credit-Agrico...,"""Zgodnie z finalnymi danymi GUS inflacja CPI w...",2022-01-17 09:20:00,pat/ asa/,PAP Biznes,bankier
298,Pierwszy obligacyjny ETF zadebiutował w ponied...,Pierwszy obligacyjny ETF zadebiutował w ponied...,Na głównym rynku GPW zadebiutował w poniedział...,Na głównym rynku GPW zadebiutował w poniedział...,https://www.bankier.pl/wiadomosc/Pierwszy-obli...,"""To już dziewiąty fundusz z rodziny Beta, a je...",2022-01-17 09:56:00,pr/ ana/,PAP Biznes,bankier


In [11]:
timestamp = int((datetime.now(timezone.utc)).timestamp())

In [12]:
bankier_dataframe.to_parquet(f'{data_dir}/{portal_name}_{timestamp}.parquet')
bankier_dataframe.to_json(f'{data_dir}/{portal_name}_{timestamp}.json')

Krótki test - dane pierwszego artykułu

In [13]:
print(f'title:\n  {bankier_dataframe.iloc[0]["title"]}')
print(f'lead_text:\n  {bankier_dataframe.iloc[0]["lead_text"]}')
print(f'lead_text_int:\n  {bankier_dataframe.iloc[0]["lead_text_int"]}')
print(f'link:\n  {bankier_dataframe.iloc[0]["link"]}')
print(f'text:\n  {bankier_dataframe.iloc[0]["text"]}')
print(f'when_published:\n  {bankier_dataframe.iloc[0]["when_published"]}')

title:
  Spadki na Wall Street koniec sesji. Nasdaq znów mocno w dół
lead_text:
  Środowa sesja na Wall Street zakończyła się spadkami pomimo wzrostów w ciągu dnia. Rynki śledzą napływające raporty finansowe spółek za czwarty kwartał, ale nadal główne obawy związane są z perspektywą zbliżających się podwyżek stóp... Czytaj dalej
lead_text_int:
  Środowa sesja na Wall Street zakończyła się spadkami pomimo wzrostów w ciągu dnia. Rynki śledzą napływające raporty finansowe spółek za czwarty kwartał, ale nadal główne obawy związane są z perspektywą zbliżających się podwyżek stóp procentowych w USA.
link:
  https://www.bankier.pl/wiadomosc/Wall-Street-w-dol-pod-koniec-sesji-Nasdaq-znow-mocno-w-dol-8262642.html
text:
  Dow Jones Industrial na zamknięciu spadł o 339 punktów, czyli 0,96 proc., do 35.028,65 pkt. S&P 500 na koniec dnia stracił 0,97 proc. i wyniósł 4.532,76 pkt. Nasdaq Composite zniżkował 1,15 proc. i zamknął sesję na poziomie 14.340,25 pkt. Indeks spadł już 10 proc. z historyczne

In [14]:
print(f'title:\n  {bankier_dataframe.iloc[99]["title"]}')
print(f'lead_text:\n  {bankier_dataframe.iloc[99]["lead_text"]}')
print(f'link:\n  {bankier_dataframe.iloc[99]["link"]}')
print(f'text:\n  {bankier_dataframe.iloc[99]["text"]}')
print(f'when_published:\n  {bankier_dataframe.iloc[99]["when_published"]}')

title:
  Słona kara za straty. Kolejna pułapka "Polskiego ładu"
lead_text:
  Nierentowna spółka zapłaci wyższy podatek niż osiągająca dochody. Trudno jej będzie go uniknąć – pisze w środę "Rzeczpospolita". Czytaj dalej
link:
  https://www.bankier.pl/wiadomosc/Slona-kara-za-straty-Kolejna-pulapka-Polskiego-ladu-8261913.html
text:
  "Polski ład" zmienia warunki prowadzenia biznesu w wielu obszarach. Trudny rok czeka nie tylko podatników PIT, ale też CIT. Spółki, które nie osiągną określonej rentowności, zapłacą po zakończeniu 2022 r. nowy podatek minimalny. To rozwiązanie, które miało być batem na podatkowych oszustów, uderzy w firmy prowadzące zwykłą działalność – czytamy w dzienniku. "Rzeczpospolita" wskazuje, że spółki zapłacą nowy podatek w dwóch przypadkach. "Po pierwsze, jeśli poniosą straty. A po drugie, gdy ich dochody będą niższe niż 1 proc. przychodów" – zaznaczy gazeta. "To szkodliwa regulacja. W praktyce może objąć nawet małych podatników CIT i będzie dodatkowym obciążeniem w

In [15]:
print(f'title:\n  {bankier_dataframe.iloc[5]["title"]}')
print(f'lead_text:\n  {bankier_dataframe.iloc[5]["lead_text"]}')
print(f'link:\n  {bankier_dataframe.iloc[5]["link"]}')
print(f'text:\n  {bankier_dataframe.iloc[5]["text"]}')
print(f'when_published:\n  {bankier_dataframe.iloc[5]["when_published"]}')

title:
  Novak Djokovic właścicielem 80 procent akcji firmy pracującej nad lekiem na COVID-19
lead_text:
  Słynny serbski tenisista Novak Djokovic jest właścicielem 80 procent akcji w duńskiej firmie biotechnologicznej QuantBioRes, która zajmuje się m.in. opracowaniem leku na COVID-19 - poinformował jej dyrektor generalny Ivan Loncarevic. Czytaj dalej
link:
  https://www.bankier.pl/wiadomosc/Novak-Djokovic-wlascicielem-80-procent-akcji-firmy-pracujacej-nad-lekiem-na-COVID-19-8262585.html
text:
  Jak przekazał, lider światowego rankingu tenisistów głównym udziałowcem został w czerwcu 2020 roku. Nie zdradził jednak kwoty, jaką Serb zainwestował w firmę. Według Loncarevica, QuantBioRes zatrudnia nieco ponad 10 naukowców pracujących w Danii, Australii i Słowenii, którzy obecnie zajmują się opracowaniem skutecznego leku na COVID-19, który jednak nie będzie szczepionką. Lek ma bazować na peptydach, naturalnych związkach chemicznych, które powstrzymywać będą koronawirusa przed infekowaniem ko